# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [15]:
# Your code here
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

cur.execute("""SELECT * FROM offices;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
2,3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
3,4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,,,France,75017,EMEA
4,5,Tokyo,+81 33 224 5000,4-1 Kioicho,,Chiyoda-Ku,Japan,102-8578,Japan
5,6,Sydney,+61 2 9264 2451,5-11 Wentworth Avenue,Floor #2,,Australia,NSW 2010,APAC
6,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA
7,27,Boston,+1 977 299 8345,105 Cambridge Street,,MA,USA,02331,NA


## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [8]:
# Your code here
cur.execute("""SELECT firstName, lastName
               FROM offices 
               JOIN employees
               WHERE city = 'Boston'
               ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()



,firstName,lastName
0,Diane,Murphy
1,Diane,Murphy
2,Mary,Patterson
3,Mary,Patterson
4,Jeff,Firrelli


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [23]:
# Your code here
cur.execute("""SELECT o.officeCode, o.city, COUNT(e.employeeNumber) AS n_employees
               FROM offices AS o
               LEFT JOIN employees AS e
               USING(officeCode)
               GROUP BY officeCode
               HAVING n_employees = 0;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,officeCode,city,n_employees
0,27,Boston,0


## Write 3 Questions of your own and answer them

In [25]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold
cur.execute("""SELECT customerName
               FROM customers
               ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df


,customerName
0,Atelier graphique
1,Signal Gift Stores
2,"Australian Collectors, Co."
3,La Rochelle Gifts
4,Baane Mini Imports
...,...
117,Motor Mint Distributors Inc.
118,Signal Collectibles Ltd.
119,"Double Decker Gift Stores, Ltd"
120,Diecast Collectables


In [26]:
# Your code here
# Which Customer has paid the most on a single order?
cur.execute("""SELECT customerName, amount
               FROM customers
               LEFT JOIN payments
               ORDER BY amount DESC
               LIMIT 1;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,customerName,amount
0,Atelier graphique,120166.58


In [34]:
# Your code here
# Who was their Sales Rep?
cur.execute("""SELECT e.employeeNumber, e.firstName, e.lastName 
               FROM employees AS e
               LEFT JOIN customers AS c
               WHERE customerName = 'Atelier graphique' 
               AND e.employeeNumber = c.salesRepEmployeeNumber;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df


,employeeNumber,firstName,lastName
0,1370,Gerard,Hernandez


In [40]:
# Your code here
# What other customers does this employee have?
cur.execute("""SELECT c.customerName
               FROM customers AS c
               LEFT JOIN employees AS e
               WHERE e.firstName = 'Gerard' AND e.lastName = 'Hernandez'
               AND c.salesRepEmployeeNumber = 1370;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,customerName
0,Atelier graphique
1,La Rochelle Gifts
2,Euro+ Shopping Channel
3,Daedalus Designs Imports
4,Mini Caravy
5,Alpha Cognac
6,Auto Associés & Cie.


## Level Up: Display the names of every individual product that each employee has sold

In [41]:
# Your code here
cur.execute("""SELECT firstName, lastName, productName
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o
               USING(customerNumber)
               JOIN orderdetails od
               USING(orderNumber)
               JOIN products p
               USING(productCode)""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head()

## NOTE: This question could also be answered using a HAVING clause which you learned about previously 
## Remember, HAVING clauses are filters similar to the WHERE clause but are conditions applied after a GROUP BY.

2996


,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang


## Level Up: Display the Number of Products each employee has sold

In [42]:
# Your code here
cur.execute("""SELECT firstName, lastName, COUNT(productName)
               FROM employees e
               JOIN customers c                   
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o 
               USING(customerNumber)
               JOIN orderdetails od 
               USING(orderNumber)                    
               JOIN products p 
               USING(productCode)
               GROUP BY lastName
               ORDER BY firstName""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df

## NOTE: Another way to access this information would be to use the dataframe from 
## the previous "level up" question and call a pandas .groupby method like so:
# df.groupby(['firstName','lastName']).count()

15


,firstName,lastName,COUNT(productName)
0,Andy,Fixter,185
1,Barry,Jones,220
2,Foon Yue,Tseng,142
3,George,Vanauf,211
4,Gerard,Hernandez,396
5,Julie,Firrelli,124
6,Larry,Bott,236
7,Leslie,Jennings,331
8,Leslie,Thompson,114
9,Loui,Bondur,177


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!